# Baseline Model

## Multiple Linear Regression:
As a baseline, we trained a multiple linear regression model using tf-idf matrices ($W_{N\times I}$) as predictors.

Note here that the $W_{N\times I}$ is a huge (5000 - 16000 columns) sparse matrix.

## Results
We split the data into training set (90%) and testing set (10%) and evaluated the model on how well the model crated by training performed on testing set.

The image below is the scatter plot of actual Score vs predicted score.
A perfect model will result in every point on diagonal line $y=x$.

We calculated Spearman's rank correlation coefficient ($\rho$), which is a measure of rank correlation (monotonic relationships between $x$ and $y$.
<img src='figures/baseline.png'>

Although better than randomly guessing, the baseline model performed poorly.
- Some points in the graph is very distant from the diagonal, perfect-model line.

- Spearman's rank correlation coefficients ranged ($\rho$) from 0.25 to 0.65.

  According to Cohen(1988)'s guidelines,
    - $0.1 < \rho < 0.3$ : small correlation
    - $0.3 < \rho < 0.5$ : medium/moderate correlation
    - $0.5 < \rho$ : large/strong correlation
    
In order to improve the model, we will next perform dimension reduction and add in a number of meta-features (e.g. essay length) that may correlate to the score.